In [48]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pyperclip
from konlpy.tag import Okt
import nltk
 
# 네이버 로그인 함수
def naver_login(driver, username, password):
    driver.get('https://nid.naver.com/nidlogin.login?mode=form&url=https://www.naver.com/')
    time.sleep(2)
    id_input = driver.find_element(By.CSS_SELECTOR, '#id')
    pyperclip.copy(username)
    id_input.send_keys(Keys.CONTROL, 'v')
    time.sleep(1)
    pw_input = driver.find_element(By.CSS_SELECTOR, '#pw')
    pyperclip.copy(password)
    pw_input.send_keys(Keys.CONTROL, 'v')
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, '#log\.login').click()
    time.sleep(2)
 
# 크롤링 함수
def crawl_naver_cafe_posts(username, password, num_pages, num_posts_per_page):
    driver = webdriver.Chrome()
    # 네이버 로그인
    naver_login(driver, username, password)
    # 다이렉트 카페 이동
    driver.get('https://cafe.naver.com/directwedding')
    time.sleep(2)
    # 검색어 입력 가전
    search = driver.find_element(By.CSS_SELECTOR, 'input#topLayerQueryInput')
    search.send_keys("중고")
    search.send_keys(Keys.RETURN)
    time.sleep(2)
    posts_data = []
    try:
        # iframe으로 전환
        driver.switch_to.frame("cafe_main")
        for page in range(1, num_pages + 1):
            for i in tqdm(range(num_posts_per_page)):
                try:
                    # 각 게시글 클릭
                    post_xpath = f'//*[@id="main-area"]/div[5]/table/tbody/tr[{i+1}]/td[1]/div[2]/div/a[1]'
                    post_link = driver.find_element(By.XPATH, post_xpath)
                    post_link.click()
                    time.sleep(1)
                    # 게시글 정보 수집
                    cont_url = driver.find_element(By.XPATH, '//*[@id="spiButton"]').get_attribute('data-url')
                    cont_date = driver.find_element(By.CLASS_NAME, 'date').text
                    cont_title = driver.find_element(By.CLASS_NAME, 'title_text').text
                    cont_text = driver.find_element(By.CLASS_NAME, 'se-main-container').text
                    posts_data.append({
                        'title': cont_title,
                        'contents': cont_text,
                        'url': cont_url,
                        'date': cont_date
                    })
                    # 뒤로 가기 (게시글 목록으로 돌아가기)
                    driver.execute_script("window.history.go(-1)")
                    time.sleep(2)
                except Exception as e:
                    print(f"Error in scraping content: {e}")
                    driver.execute_script("window.history.go(-1)")
                    time.sleep(2)
            # 다음 페이지로 이동
            try:
                if page % 10 == 0:  # 매 10페이지마다 '다음' 버튼 클릭
                    next_button = driver.find_element(By.LINK_TEXT, '다음')
                    next_button.click()
                else:
                    page_link = driver.find_element(By.LINK_TEXT, str(page + 1))
                    page_link.click()
                time.sleep(2)
            except Exception as e:
                print(f"Error in navigating to the next page: {e}")
                break
    except Exception as e:
        print(f"Error in crawling: {e}")
    finally:
        driver.quit()
    # 데이터프레임 생성
    df = pd.DataFrame(posts_data)
    return df

def filter_positive_posts(df):
    okt = Okt()
    positive_words_ko = ['냉장고', '중고가전','세탁기','공기청정기','에어컨','건조기','티비','텔레비전','공청기','워시타워','식세기']
    positive_words_en = ['used', 'tv']
    filtered_posts = []

    for index, row in df.iterrows():
        title_morphs_ko = okt.morphs(row['title'])
        contents_morphs_ko = okt.morphs(row['contents'])
        title_words_en = nltk.word_tokenize(row['title'])
        contents_words_en = nltk.word_tokenize(row['contents'])
        
        if (any(word in title_morphs_ko for word in positive_words_ko) or
            any(word in contents_morphs_ko for word in positive_words_ko) or
            any(word in title_words_en for word in positive_words_en) or
            any(word in contents_words_en for word in positive_words_en)):
            filtered_posts.append(row)

    filtered_df = pd.DataFrame(filtered_posts)
    return filtered_df


# 크롤링 실행
username = "lisa3472"
password = "lee0919hj"
num_pages = 20  # Adjust based on how many pages you want to scrape
num_posts_per_page = 15  # Adjust based on how many posts per page you want to scrape
df = crawl_naver_cafe_posts(username, password, num_pages, num_posts_per_page)
# 데이터프레임 출력
df.to_csv('directwedding.csv', encoding='utf-8-sig', index=False)

# 긍정어 포함된 데이터프레임 출력
filtered_df = filter_positive_posts(df)
#filtered_df.to_csv('filtered_directwedding.csv', encoding='utf-8-sig', index=False)

100%|██████████| 15/15 [00:49<00:00,  3.33s/it]


In [49]:
filtered_df

,title,contents,url,date
3,다들 가전 몇 가지 구매하셨나요??ㅎㅎ,"가전제품 알아보면 알아볼 수록 결정이 어려워요..\n특히 3품목,6품목,9품목 많이...",https://cafe.naver.com/directwedding/6672889,2024.07.09. 14:52
8,여러분들이라면 어떻게 하실건가요? (신혼 가전제품 관련),안녕하세요 내년에 결혼 예정인 예신인데요\n신혼집 관련해서 예랑쪽 회사에 사택복지가...,https://cafe.naver.com/directwedding/6669145,2024.07.08. 14:12
10,중소 티비 사자는 신랑 맞는걸까요..,다른가는 lg로 맞추고 티비는 나중에 생길 아기 때문에 깨질수 있다고 중소티비 하자...,https://cafe.naver.com/directwedding/6668355,2024.07.08. 11:07
12,LG 전자 롯데백화점 부산본점 모재경 명예대명장님께 가전 졸업한 후기입니다.,안녕하세요! 8월 입주 예정이고 어제 가전 졸업한 후기입니다!\n제가 생각했던 것보...,https://cafe.naver.com/directwedding/6668134,2024.07.08. 10:20
22,도와주세요.. 알고보니 빚쟁이 예비 남편,안녕하세요\n일단 예비남편이랑은 동거중이고요 .. 결혼을 약속한 사이인 20대 예비...,https://cafe.naver.com/directwedding/6661910,2024.07.05. 15:22
28,이사 집주인 월세 조언 부탁드립니다 ㅠㅠ,"안녕하세요 월세살이 집주인 마찰이 있어서 ,현명하신 분들의 조언을 얻고자 글을 쓰게...",https://cafe.naver.com/directwedding/6657475,2024.07.04. 09:56
29,신혼집 가전 어떻게 하시나요?,"안녕하세요 내년 3월 예식을 앞둔 예신입니다~\n결혼 전 동거를 하게 되면서 가전,...",https://cafe.naver.com/directwedding/6653561,2024.07.03. 08:51
31,에어컨 설치비,이사할때 에어컨 철거 시 콘센트 코드 있는 부분도 다 자르나요?\n2년쓰고 이사했는...,https://cafe.naver.com/directwedding/6652620,2024.07.02. 21:06
34,"신혼가전 당근으로 샀는데, 환불 되나요...","5/2에 당근으로 에어컨을 샀는데, 19년식 2in1 LG에어컨이고 80에 올라온 ...",https://cafe.naver.com/directwedding/6649685,2024.07.02. 10:17
47,삼성전자 비스포크 인덕션 NZ63B6507XL 후기입니다!,안녕하세요\n가전 시리즈 벌써 세번째 글입니다.\nLG전자 워시타워(세탁기 건조기)...,https://cafe.naver.com/directwedding/6641247,2024.06.30. 20:43


In [23]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 3 to 12
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     4 non-null      object
 1   contents  4 non-null      object
 2   url       4 non-null      object
 3   date      4 non-null      object
dtypes: object(4)
memory usage: 160.0+ bytes


In [50]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup as bs


# 로그인에 필요한 정보
naver_id = 'lisa3472'
naver_pw = 'lee0919hj'

driver = webdriver.Chrome()

# 네이버 로그인 페이지 URL
login_url = 'https://nid.naver.com/nidlogin.login'

# 로그인 함수 정의
def login_naver():
    driver.get(login_url)
    time.sleep(3)

    id_input = driver.find_element(By.CSS_SELECTOR, '#id')
    pyperclip.copy(naver_id)
    id_input.send_keys(Keys.CONTROL, 'v')
    time.sleep(1)
    pw_input = driver.find_element(By.CSS_SELECTOR, '#pw')
    pyperclip.copy(naver_pw)
    pw_input.send_keys(Keys.CONTROL, 'v')
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, '#log\.login').click()
    time.sleep(2)

# iframe 내의 댓글을 추출하는 함수 정의
def extract_comments_in_iframe(url):
    try:
        driver.get(url)
        time.sleep(5)  # 페이지가 로딩될 때까지 잠시 대기
        
        # iframe으로 전환
        iframe_element = driver.find_element(By.CSS_SELECTOR, 'iframe#cafe_main')
        driver.switch_to.frame(iframe_element)
        
        # iframe 내의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # 댓글을 포함한 요소를 선택하여 텍스트 추출
        comments = driver.find_elements(By.CLASS_NAME, 'comment_text_view')
        comment_texts = [comment.text.strip() for comment in comments]

        return comment_texts
    except Exception as e:
        print(f'Error accessing URL: {e}')
        return None

try:
    # 네이버에 로그인
    login_naver()

    # 각 행의 url을 순회하면서 댓글을 추출하여 리스트에 추가
    comments_list = []
    for idx, row in filtered_df.iterrows():
        url = row['url']
        comments = extract_comments_in_iframe(url)
        if comments:
            for comment in comments:
                comments_list.append({'url': url, 'comments': comment})

    # 댓글 리스트를 데이터프레임으로 변환
    comments_df = pd.DataFrame(comments_list)

    # 결과 데이터프레임 출력
    print(comments_df)

finally:
    # WebDriver 종료
    driver.quit()

                                               url  \
0     https://cafe.naver.com/directwedding/6672889   
1     https://cafe.naver.com/directwedding/6672889   
2     https://cafe.naver.com/directwedding/6672889   
3     https://cafe.naver.com/directwedding/6672889   
4     https://cafe.naver.com/directwedding/6672889   
...                                            ...   
1379  https://cafe.naver.com/directwedding/6490117   
1380  https://cafe.naver.com/directwedding/6490117   
1381  https://cafe.naver.com/directwedding/6490117   
1382  https://cafe.naver.com/directwedding/6490117   
1383  https://cafe.naver.com/directwedding/6490117   

                                               comments  
0           보통 6개 (냉장고 에어컨 티비 건조기 세탁기 청소기) 많이들 하시는거같아요!  
1       두분이서만 사실거면 김치냉장고까지 필요없어서 예랑님 말대로 하시면 괜찮을 것 같아요!  
2     9개가 혜택이 크긴 하지만 안쓰는 제품넣어서 굳이 혜택받는것 보다는 쓰는것만 하는게...  
3         저는 금액대별 할인이어서 트윈원 에어컨, 티비, 세탁기, 청소기, 냉장고 했어요!  
4                                저희는 5품목 딱 필요한 것만 했어요ㅎㅎ  
...

In [55]:
comments_df

,url,comments
0,https://cafe.naver.com/directwedding/6672889,보통 6개 (냉장고 에어컨 티비 건조기 세탁기 청소기) 많이들 하시는거같아요!
1,https://cafe.naver.com/directwedding/6672889,두분이서만 사실거면 김치냉장고까지 필요없어서 예랑님 말대로 하시면 괜찮을 것 같아요!
2,https://cafe.naver.com/directwedding/6672889,9개가 혜택이 크긴 하지만 안쓰는 제품넣어서 굳이 혜택받는것 보다는 쓰는것만 하는게...
3,https://cafe.naver.com/directwedding/6672889,"저는 금액대별 할인이어서 트윈원 에어컨, 티비, 세탁기, 청소기, 냉장고 했어요!"
4,https://cafe.naver.com/directwedding/6672889,저희는 5품목 딱 필요한 것만 했어요ㅎㅎ
...,...,...
1379,https://cafe.naver.com/directwedding/6490117,인테리어 고급지고 깔끔하게 잘되셨네요 ㅎㅎ 층고 높은거도 부러워요!!
1380,https://cafe.naver.com/directwedding/6490117,우와 층고높은 신혼집이라니 고급스럽고 너무 깔끔해서 예뻐요 ㅎㅎ
1381,https://cafe.naver.com/directwedding/6490117,우와 천장이 높으니까 훠얼씬 넓어보이고 탁 트였어요!
1382,https://cafe.naver.com/directwedding/6490117,층고 높은 건 정말 좋은 것 같아요ㅎㅎ 집이 구축 같지 않고 깨끗하고 좋은데요?


In [58]:
comments_df = comments_df[comments_df['comments'].str.contains('중고')]

In [59]:
comments_df

,url,comments
15,https://cafe.naver.com/directwedding/6672889,할인 받고 구매하시고 정 필요 없으신 건 중고거래해서 자체 페이백하시는 방법도 있어...
36,https://cafe.naver.com/directwedding/6669145,이사가는 집 사이즈에 따라 사용하는 가전도 달라서 중고로 쓰시다가 이사가실 때 사셔...
41,https://cafe.naver.com/directwedding/6669145,중고가전이 생각보다 많이 저렴해요 ㅋㅋ 저도 비슷한 상황인데 세탁기 빼고는 전부 중...
44,https://cafe.naver.com/directwedding/6669145,저도 최소한의 중고 + 가전으로 꾸민후에 제 집이 생기면 마련할래요~~!\n신혼집도...
45,https://cafe.naver.com/directwedding/6669145,모두 댓글 감사합니다 :) ❤\n항상 현명한 예랑이 둬서 다행이네요 ㅠㅠ ㅋㅋㅋ\n...
...,...,...
1250,https://cafe.naver.com/directwedding/6495390,가전은 중고쓰는게 아닌것 같아요 금방 망가질라요
1267,https://cafe.naver.com/directwedding/6495390,가전은 중고는 비추에요... 발품팔아서 저렴한곳 찾아보시는게 어떨까요
1295,https://cafe.naver.com/directwedding/6495390,전세니까 딱필요한고만 매장가서 혼수로 살꼬같아요! 인터넷이랑 가격 비교 해보구용! ...
1344,https://cafe.naver.com/directwedding/6490308,나중에 필요하심 중고거래로 미개봉 새상품 같은거 사세요ㅎㅎ


In [60]:
comments_df.to_csv('wedding_co.csv', encoding='utf-8-sig', index=False)